In [2]:
import matplotlib.pyplot as plt
import re
import os
import numpy as np
from plots import plot_train_compare2, plot_train_compare

/Users/alexandermurphy/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def get_hp_args_from_txt(path):
    with open(path) as f:
        data_string = f.read()

        # Step 2: Define regex patterns to extract the values
        poison_lr_pattern = r"'poison_lr'\s*:\s*([0-9.]+)"
        iterations_pattern = r"'iterations'\s*:\s*(\d+)"
        epsilon_pattern = r"'epsilon'\s*:\s*([0-9.]+)"
        poison_start_epoch_pattern = r"'poison_start_epoch'\s*:\s*(\d+)"

        # Step 3: Extract the values using regex
        poison_lr = re.search(poison_lr_pattern, data_string).group(1)
        iterations = re.search(iterations_pattern, data_string).group(1)
        epsilon = re.search(epsilon_pattern, data_string).group(1)
        poison_start_epoch = re.search(poison_start_epoch_pattern, data_string).group(1)

        # Print the extracted values
        return (poison_lr, iterations, epsilon, poison_start_epoch)

def get_tests_vals_from_txt(path):
    vals = []
    tests = []
    vals_p = []
    tests_p = []
    with open(path) as f:
        highest_val = -1
        highest_val_p = -1
        highest_test = -1
        highest_test_p = -1
        i = 0
        for l in f.readlines():
            i += 1
            val_acc = float(re.findall(r"Val_accuracy:(\d+\.\d+)", l)[0])
            test_acc = float(re.findall(r"Test_accuracy:(\d+\.\d+)", l)[0])
            if i >= 200:
                break
            elif i >= 100:
                if val_acc > highest_val_p:
                    highest_val_p = val_acc
                    highest_test_p = test_acc
                vals_p.append(val_acc)
                tests_p.append(test_acc)
            elif i < 100:
                if val_acc > highest_val:
                    highest_val = val_acc
                    highest_test = test_acc
                vals.append(val_acc)
                tests.append(test_acc)
    return (vals, vals_p, highest_val, highest_val_p, tests, tests_p, highest_test, highest_test_p)
    
def construct_run_dicts():
    base_path = 'experiment_results_from_eddie/fmnist_res_net_18/poison_tune'
    ids = next(os.walk(base_path))[1]
    runs = {}
    for id in ids:
        d = {}
        p_args = f"{base_path}/{id}/args.txt"
        poison_lr, iterations, epsilon, poison_start_epoch = get_hp_args_from_txt(p_args)
        d["poison_lr"] = float(poison_lr)
        d["iterations"] = int(iterations)
        d["epsilon"] = float(epsilon)
        d["poison_start_epoch"] = int(poison_start_epoch)

        path = f"{base_path}/{id}/metrics.txt"
        vals, vals_p, highest_val, highest_val_p, tests, tests_p, highest_test, highest_test_p = get_tests_vals_from_txt(path)
        
        d["vals"] = vals
        d["vals_p"] = vals_p
        d["highest_val"] = highest_val
        d["highest_val_p"] = highest_val_p
        
        d["tests"] = tests
        d["tests_p"] = tests_p
        d["highest_test"] = highest_test
        d["highest_test_p"] = highest_test_p

        d["ratio"] = d["highest_val_p"]/d["highest_val"]


        runs[id] = d
    return runs

runs = construct_run_dicts()